In [2]:
import pandas as pd

In [118]:
# ################################# LOAD DATA ################################
people_to_people_df = pd.read_csv('static/data/people_to_people.csv')


In [120]:
# ############################## PREPROCESS DATA #############################
# Drop druplicates
edge_data = people_to_people_df.drop_duplicates(keep="first").reset_index(drop=True)
# Remove self-edges(edges where the source and target nodes are the same)
edge_data = edge_data[edge_data['from'] != edge_data['to']].reset_index(drop=True)

edges = edge_data.copy()
nodes = set()

In [121]:
# ################################# LOAD DATA ################################
grant_to_people_df = pd.read_csv('static/data/grants_to_people.csv')


# ############################## PREPROCESS DATA #############################
# Drop druplicates
edge_data = grant_to_people_df.drop_duplicates(keep="first").reset_index(drop=True)
# Remove self-edges(edges where the source and target nodes are the same)
edge_data = edge_data[edge_data['from'] != edge_data['to']].reset_index(drop=True)

edges_cyto = edge_data.copy()
nodes_cyto = set()

In [122]:
cyto_edges = []
cyto_nodes = []

for index, edge in edges_cyto.iterrows():
    source, target = edge['from'], edge['to']

    if source not in nodes_cyto:
        nodes_cyto.add(source)
        cyto_nodes.append({
            "data": {
                "id": str(source), 
                "label": str(source)
            }
        })
    if target not in nodes_cyto:
        nodes.add(target)
        cyto_nodes.append({
            "data": {
                "id": str(target), 
                "label": str(target)
            }
        })
        
    cyto_edges.append({
        "data": {
            "source": str(source), 
            "target": str(target)
            }
        })

In [123]:
cyto_nodes

[{'data': {'id': '77795114', 'label': '77795114'}},
 {'data': {'id': '100396', 'label': '100396'}},
 {'data': {'id': '83979631', 'label': '83979631'}},
 {'data': {'id': '100396', 'label': '100396'}},
 {'data': {'id': '30407538', 'label': '30407538'}},
 {'data': {'id': '113487', 'label': '113487'}},
 {'data': {'id': '65317644', 'label': '65317644'}},
 {'data': {'id': '113487', 'label': '113487'}},
 {'data': {'id': '22229355', 'label': '22229355'}},
 {'data': {'id': '115283', 'label': '115283'}},
 {'data': {'id': '36428007', 'label': '36428007'}},
 {'data': {'id': '115283', 'label': '115283'}},
 {'data': {'id': '95405394', 'label': '95405394'}},
 {'data': {'id': '115283', 'label': '115283'}},
 {'data': {'id': '123956', 'label': '123956'}},
 {'data': {'id': '39124998', 'label': '39124998'}},
 {'data': {'id': '123993', 'label': '123993'}},
 {'data': {'id': '64040463', 'label': '64040463'}},
 {'data': {'id': '123993', 'label': '123993'}},
 {'data': {'id': '81786442', 'label': '81786442'}},


In [101]:
def update_graph(search_value):
    if not search_value:
        return cyto_edges + cyto_nodes
    filtered_data_edges = []
    filtered_data_nodes = set()

    for edge in cyto_edges:  
        if edge['data']['source'] == search_value or edge['data']['target'] == search_value:
            filtered_data_edges.append(edge)
            filtered_data_nodes.update([edge['data']['source'], edge['data']['target']])
    new_cyto_nodes = [node for node in cyto_nodes if node['data']['id'] in filtered_data_nodes]
  
    return filtered_data_edges

In [102]:
update_graph('66003097')

[{'data': {'source': '66003097', 'target': '628813'}},
 {'data': {'source': '66003097', 'target': '990032'}}]

In [193]:
# Initialize global data structures for node expansion (followers and following)
def populate_relationships(edges: list):
    """This function creates and populates dictionaries a undirected network representing nodes and edges in the graph."""
    node_di = {}
    edges_di = {}

    # Iterate through each edge to populate following and followers dictionaries
    for index, edge in edges.iterrows():
        source = edge['from']
        target = edge['to']

        # Populate relationships
        if source not in edges_di:
            edges_di[source] = []
            

        edges_di[source].append({
            "data": {
                "id": str(source + target),
                "source": str(source),
                "target": str(target)
            }
        })

        # Populate followers relationships
        if target not in edges_di:
            edges_di[target] = []
            

        edges_di[target].append({
            "data": {
                "id": str(source + target),
                "source": str(source),
                "target": str(target)
            }
        })

    # Now populate the node list on the source or target list, which is not the node in the loop.
    for node, edges in edges_di.items():
        # Add the node itself with its data
        node_di[node] = [{'data': {'id': str(node), 'label': str(node)}}]
        # add all the nodes are from the node in question 
        node_di[node] = [{'data': {'id': str(edge['data']['source']), 'label': str(edge['data']['source'])}} for edge in edges if str(node) != edge['data']['source']]
        # add all the nodes that are to the node in question
        node_di[node] = [{'data': {'id': str(edge['data']['target']), 'label': str(edge['data']['target'])}} for edge in edges if str(node) != edge['data']['target']]
        
    return node_di, edges_di

In [197]:
def populate_relationships(edges: list):
    """This function creates and populates dictionaries for an undirected network representing nodes and edges in the graph."""
    node_di = {}
    edges_di = {}

    # Iterate through each edge to populate dictionaries
    for index, edge in edges.iterrows():
        source = edge['from']
        target = edge['to']

        # Populate edges for source node
        if source not in edges_di:
            edges_di[source] = []
        edges_di[source].append({
            "data": {
                "id": str(source + target),
                "source": str(source),
                "target": str(target)
            }
        })

        # Populate edges for target node
        if target not in edges_di:
            edges_di[target] = []
        edges_di[target].append({
            "data": {
                "id": str(source + target),
                "source": str(source),
                "target": str(target)
            }
        })

    # Now populate the node list on the source or target list, which is not the node in the loop.
    for node, edges in edges_di.items():
        # Add the node itself with its data
        node_di[node] = [{'data': {'id': str(node), 'label': str(node)}}]
        # Add all the nodes that are connected to the node in question
        for edge in edges:
            if str(node) != edge['data']['source']:
                node_di[node].append({'data': {'id': str(edge['data']['source']), 'label': str(edge['data']['source'])}})
            if str(node) != edge['data']['target']:
                node_di[node].append({'data': {'id': str(edge['data']['target']), 'label': str(edge['data']['target'])}})
        
    return node_di, edges_di


In [198]:
nodes_di, edges_di = populate_relationships(edges)

In [199]:
nodes_di[77795114]

[{'data': {'id': '77795114', 'label': '77795114'}},
 {'data': {'id': '83979631', 'label': '83979631'}},
 {'data': {'id': '90068471', 'label': '90068471'}},
 {'data': {'id': '12811381', 'label': '12811381'}},
 {'data': {'id': '18797839', 'label': '18797839'}},
 {'data': {'id': '20129124', 'label': '20129124'}},
 {'data': {'id': '44105450', 'label': '44105450'}},
 {'data': {'id': '55662086', 'label': '55662086'}},
 {'data': {'id': '58961111', 'label': '58961111'}},
 {'data': {'id': '98146154', 'label': '98146154'}},
 {'data': {'id': '27742873', 'label': '27742873'}},
 {'data': {'id': '41902096', 'label': '41902096'}},
 {'data': {'id': '42465713', 'label': '42465713'}},
 {'data': {'id': '53181317', 'label': '53181317'}},
 {'data': {'id': '79893241', 'label': '79893241'}},
 {'data': {'id': '91680829', 'label': '91680829'}},
 {'data': {'id': '92021548', 'label': '92021548'}},
 {'data': {'id': '92911885', 'label': '92911885'}},
 {'data': {'id': '97794113', 'label': '97794113'}},
 {'data': {'

In [182]:
edges_di

{77795114: [{'data': {'id': '161774745',
    'source': '77795114',
    'target': '83979631'}},
  {'data': {'id': '167863585', 'source': '77795114', 'target': '90068471'}},
  {'data': {'id': '90606495', 'source': '12811381', 'target': '77795114'}},
  {'data': {'id': '96592953', 'source': '18797839', 'target': '77795114'}},
  {'data': {'id': '97924238', 'source': '20129124', 'target': '77795114'}},
  {'data': {'id': '121900564', 'source': '44105450', 'target': '77795114'}},
  {'data': {'id': '133457200', 'source': '55662086', 'target': '77795114'}},
  {'data': {'id': '136756225', 'source': '58961111', 'target': '77795114'}},
  {'data': {'id': '175941268', 'source': '77795114', 'target': '98146154'}},
  {'data': {'id': '105537987', 'source': '27742873', 'target': '77795114'}},
  {'data': {'id': '119697210', 'source': '41902096', 'target': '77795114'}},
  {'data': {'id': '120260827', 'source': '42465713', 'target': '77795114'}},
  {'data': {'id': '130976431', 'source': '53181317', 'target'

In [175]:
edges_di[77795114]

[{'data': {'id': '161774745', 'source': '77795114', 'target': '83979631'}},
 {'data': {'id': '167863585', 'source': '77795114', 'target': '90068471'}},
 {'data': {'id': '90606495', 'source': '12811381', 'target': '77795114'}},
 {'data': {'id': '96592953', 'source': '18797839', 'target': '77795114'}},
 {'data': {'id': '97924238', 'source': '20129124', 'target': '77795114'}},
 {'data': {'id': '121900564', 'source': '44105450', 'target': '77795114'}},
 {'data': {'id': '133457200', 'source': '55662086', 'target': '77795114'}},
 {'data': {'id': '136756225', 'source': '58961111', 'target': '77795114'}},
 {'data': {'id': '175941268', 'source': '77795114', 'target': '98146154'}},
 {'data': {'id': '105537987', 'source': '27742873', 'target': '77795114'}},
 {'data': {'id': '119697210', 'source': '41902096', 'target': '77795114'}},
 {'data': {'id': '120260827', 'source': '42465713', 'target': '77795114'}},
 {'data': {'id': '130976431', 'source': '53181317', 'target': '77795114'}},
 {'data': {'id'

In [176]:
nodes_di[77795114]

[{'data': {'id': '83979631', 'label': '83979631'}},
 {'data': {'id': '90068471', 'label': '90068471'}},
 {'data': {'id': '98146154', 'label': '98146154'}},
 {'data': {'id': '79893241', 'label': '79893241'}},
 {'data': {'id': '91680829', 'label': '91680829'}},
 {'data': {'id': '92021548', 'label': '92021548'}},
 {'data': {'id': '92911885', 'label': '92911885'}},
 {'data': {'id': '97794113', 'label': '97794113'}},
 {'data': {'id': '88352127', 'label': '88352127'}},
 {'data': {'id': '85478689', 'label': '85478689'}},
 {'data': {'id': '87243668', 'label': '87243668'}},
 {'data': {'id': '90326364', 'label': '90326364'}},
 {'data': {'id': '92452327', 'label': '92452327'}},
 {'data': {'id': '98229284', 'label': '98229284'}},
 {'data': {'id': '98473661', 'label': '98473661'}}]

In [46]:
edges_di

{77795114: [{'data': {'id': '161774745',
    'source': '77795114',
    'target': '83979631'}},
  {'data': {'id': '167863585', 'source': '77795114', 'target': '90068471'}},
  {'data': {'id': '90606495', 'source': '12811381', 'target': '77795114'}},
  {'data': {'id': '96592953', 'source': '18797839', 'target': '77795114'}},
  {'data': {'id': '97924238', 'source': '20129124', 'target': '77795114'}},
  {'data': {'id': '121900564', 'source': '44105450', 'target': '77795114'}},
  {'data': {'id': '133457200', 'source': '55662086', 'target': '77795114'}},
  {'data': {'id': '136756225', 'source': '58961111', 'target': '77795114'}},
  {'data': {'id': '175941268', 'source': '77795114', 'target': '98146154'}},
  {'data': {'id': '105537987', 'source': '27742873', 'target': '77795114'}},
  {'data': {'id': '119697210', 'source': '41902096', 'target': '77795114'}},
  {'data': {'id': '120260827', 'source': '42465713', 'target': '77795114'}},
  {'data': {'id': '130976431', 'source': '53181317', 'target'

In [96]:
for node in nodes_di[77795114]:
    print(node['data']['id'])

77795114
77795114
12811381
18797839
20129124
44105450
55662086
58961111
77795114
27742873
41902096
42465713
53181317
77795114
77795114
77795114
77795114
77795114
77795114
13561646
15259445
17300961
21648930
23820000
24215729
29705789
30327814
30483914
31512608
33868824
33943990
41088550
42207744
48094348
50243699
52125232
52811724
55797810
56185396
57943081
60245706
62144110
62439051
63276540
70247785
75384546
76282315
77795114
77795114
77795114
77795114
77795114
77795114
51653897
56721313


In [49]:
for data in nodes_di[82126353]:
    print(data['data']['id'])

16011892
34513296
53414185
82126353


In [22]:
for data in edges_di[50984631]:
    print(data['data']['source'])

17300961
19242451
19638872
22273594
22372749
27329343
28738958
28916522
29511878
30621101
33359265
37784142
40070800
40326129
41579687
44532410
45392482
45986830
47459630
48119858
49991169
50070388
50632751
50984631
50984631
50984631
50984631
50984631
50984631
50984631
50984631
50984631
50984631
50984631
50984631
50984631
50984631
